In [2]:
using LaTeXStrings

In [3]:
module TempModel
    data_dir="../"
    using GaussianProcesses
    using GaussianProcesses: set_params!
    include(data_dir*"/src/fitted_kernel.jl")
end

TempModel

In [4]:
param_dict = Dict{String,Dict{String,String}}() # store parameter values

Dict{String,Dict{String,String}} with 0 entries

In [5]:
let
    kern = "SExSE"
    param_dict[kern] = Dict{AbstractString,String}()
    
    k, logNoise = TempModel.fitted_sptemp_SExSE()

    k_time, k_space = k.kerns[1].kerns

    σ_eps   = exp(logNoise)
    σ_time  = √(k_time.kern.σ2 * k_space.kern.kern.σ2)
    ℓ_time  = √(k_time.kern.ℓ2)
    ℓ_space = √(k_space.kern.kern.ℓ2)
    param_dict[kern]["σ_eps"] = @sprintf("%.1f", σ_eps)
    param_dict[kern]["σ_time"] = @sprintf("%.1f", σ_time)
    param_dict[kern]["ℓ_time"] = @sprintf("%.1f", ℓ_time)
    param_dict[kern]["ℓ_space"] = @sprintf("%.0f", ℓ_space/1000)
end
print(param_dict)

Dict("SExSE"=>Dict("σ_eps"=>"0.4","σ_time"=>"3.7","ℓ_space"=>"176","ℓ_time"=>"2.7"))

In [6]:
let
    kern = "diurnal"
    param_dict[kern] = Dict{AbstractString,String}()

    k, logNoise =  TempModel.fitted_sptemp_diurnal()

    σ_eps   = exp(logNoise)

    k_spt, k_24, k_mu = k.kerns

    k_time, k_space = k_spt.kerns

    k_t24, k_s24 = k_24.kerns

    σ_eps   = exp(logNoise)
    σ_time  = √(k_time.kern.σ2 * k_space.kern.kern.σ2)
    ℓ_time  = √(k_time.kern.ℓ2)
    ℓ_space = √(k_space.kern.kern.ℓ2)

    σ_24 = √(k_t24.kern.kern.σ2 * k_s24.kern.kern.σ2)
    ℓ_24 = √(k_t24.kern.kern.ℓ2)
    ℓ_s24 = √(k_s24.kern.kern.ℓ2)

    param_dict[kern]["σ_eps"] = @sprintf("%.1f", σ_eps)
    param_dict[kern]["σ_time"] = @sprintf("%.1f", σ_time)
    param_dict[kern]["ℓ_time"] = @sprintf("%.1f", ℓ_time)
    param_dict[kern]["ℓ_space"] = @sprintf("%.0f", ℓ_space/1000)
    param_dict[kern]["σ_24"] = @sprintf("%.1f", σ_24)
    param_dict[kern]["ℓ_24"] = @sprintf("%.1f", ℓ_24)
    param_dict[kern]["ℓ_s24"] = @sprintf("%.0f", ℓ_s24/1000)
end

"1414"

In [56]:
let
    kern = "sumprod"
    param_dict[kern] = Dict{AbstractString,String}()
    k, logNoise =  TempModel.fitted_sptemp_simpler()

    k_24, k2, k1, k3, k_mu = k.kerns
    
    σ_eps   = exp(logNoise)
    param_dict[kern]["σ_eps"] = @sprintf("%.1f", σ_eps)
    
    k_t24, k_s24 = k_24.kerns
    σ_24 = √(k_t24.kern.kern.σ2 * k_s24.kern.σ2)
    ℓ_24 = √(k_t24.kern.kern.ℓ2)
    ℓ_s24 = √(k_s24.kern.ℓ2)
    param_dict[kern]["σ_24"] = @sprintf("%.1f", σ_24)
    param_dict[kern]["ℓ_24"] = @sprintf("%.1f", ℓ_24)
    param_dict[kern]["ℓ_s24"] = @sprintf("%.0f", ℓ_s24/1000)

    for (i,k_spt) in enumerate((k1, k2, k3))
        k_time, k_space = k_spt.kerns
        σ_time  = √(k_time.kern.σ2 * k_space.kern.σ2)
        α_time = (k_time.kern.α)
        ℓ_time  = √(k_time.kern.ℓ2)
        ℓ_space = √(k_space.kern.ℓ2)

        param_dict[kern]["σ_time"*string(i)] = @sprintf("%.1f", σ_time)
        param_dict[kern]["ℓ_time"*string(i)] = @sprintf("%.1f", ℓ_time)
        param_dict[kern]["ℓ_space"*string(i)] = @sprintf("%.0f", ℓ_space/1000)
        param_dict[kern]["α_time"*string(i)] = @sprintf("%.1f", α_time)
    end
end

In [79]:
kerns = ["SExSE", "diurnal", "sumprod"]
latex_kern = Dict(
    "SExSE" => L"\kSESE",
    "diurnal" => L"\kdiurn",
    "sumprod" => L"\ksumprod",
    )
rows = [
    "σ_eps",
    "\\hline",
    "σ_time",
    "ℓ_time",
    "ℓ_space",
    "\\hline",
    "σ_24",
    "ℓ_24",
    "ℓ_s24",
    "\\hline",
    "σ_time1",
    "ℓ_time1",
    "ℓ_space1",
    "α_time1",
    "\\hline",
    "σ_time2",
    "ℓ_time2",
    "ℓ_space2",
    "α_time2",
    "\\hline",
    "σ_time3",
    "ℓ_time3",
    "ℓ_space3",
    "α_time3",
]
latex_param = Dict(
    "σ_eps"   => L"\sigman",
    "σ_time"  => L"\sigma_\subtime",
    "ℓ_time"  => L"\ell_\subtime",
    "ℓ_space" => L"\ell_\subspace",
    "σ_24"    => L"\sigma_{24}",
    "ℓ_24"    => L"\ell_{24}",
    "ℓ_s24"   => L"\ell_{\subspace{}24}",
    "σ_time1"  => L"\sigma_{\subtime 1}",
    "ℓ_time1"  => L"\ell_{\subtime 1}",
    "ℓ_space1" => L"\ell_{\subspace 1}",
    "σ_time2"  => L"\sigma_{\subtime 2}",
    "ℓ_time2"  => L"\ell_{\subtime 2}",
    "ℓ_space2" => L"\ell_{\subspace 2}",
    "σ_time3"  => L"\sigma_{\subtime 3}",
    "ℓ_time3"  => L"\ell_{\subtime 3}",
    "ℓ_space3" => L"\ell_{\subspace 3}",
    "α_time" => L"\alpha_{\subtime}",
    "α_time1" => L"\alpha_{\subtime 1}",
    "α_time2" => L"\alpha_{\subtime 2}",
    "α_time3" => L"\alpha_{\subtime 3}",
    )
units = Dict(
    "σ_eps"   => L"$\degreeC$",
    "σ_time"  => L"$\degreeC$",
    "ℓ_time"  => L"$\mathrm{hr}$",
    "ℓ_space" => L"$\mathrm{km}$",
    "σ_time1"  => L"$\degreeC$",
    "ℓ_time1"  => L"$\mathrm{hr}$",
    "ℓ_space1" => L"$\mathrm{km}$",
    "σ_time2"  => L"$\degreeC$",
    "ℓ_time2"  => L"$\mathrm{hr}$",
    "ℓ_space2" => L"$\mathrm{km}$",
    "σ_time3"  => L"$\degreeC$",
    "ℓ_time3"  => L"$\mathrm{hr}$",
    "ℓ_space3" => L"$\mathrm{km}$",
    "σ_24"    => L"$\degreeC$",
    "ℓ_24"    => L"$\mathrm{hr}$",
    "ℓ_s24"   => L"$\mathrm{km}$",
    "α_time" => "",
    "α_time1" => "",
    "α_time2" => "",
    "α_time3" => "",
    )

Dict{String,Any} with 20 entries:
  "α_time2"  => ""
  "σ_time2"  => L"$\degreeC$"
  "α_time3"  => ""
  "σ_time"   => L"$\degreeC$"
  "ℓ_time2"  => L"$\mathrm{hr}$"
  "ℓ_space2" => L"$\mathrm{km}$"
  "α_time"   => ""
  "σ_time3"  => L"$\degreeC$"
  "ℓ_24"     => L"$\mathrm{hr}$"
  "ℓ_time1"  => L"$\mathrm{hr}$"
  "ℓ_time3"  => L"$\mathrm{hr}$"
  "ℓ_s24"    => L"$\mathrm{km}$"
  "σ_time1"  => L"$\degreeC$"
  "σ_eps"    => L"$\degreeC$"
  "α_time1"  => ""
  "ℓ_space"  => L"$\mathrm{km}$"
  "σ_24"     => L"$\degreeC$"
  "ℓ_space1" => L"$\mathrm{km}$"
  "ℓ_time"   => L"$\mathrm{hr}$"
  "ℓ_space3" => L"$\mathrm{km}$"

In [75]:
print("l")
print("|")
for param in rows
    if param == "\\hline"
        print("|")
        continue
    end
    print("r")
end
print("\n")
for param in rows
    if param == "\\hline"
        continue
    end
    print(" & ")
    print("\\rotcolhead{")
    print(latex_param[param])
    print("}")
end
print("\\\\ \n")
print("\\hline\n")
for kern in kerns
    print(latex_kern[kern])
    for param in rows
        if param == "\\hline"
            continue
        end
        print(" & ")
        print(get(param_dict[kern], param, ""))        
    end
    print("\\\\ \n")
end

l|r|rrr|rrr|rrrr|rrrr|rrrr
 & \rotcolhead{$\sigman$} & \rotcolhead{$\sigma_\subtime$} & \rotcolhead{$\ell_\subtime$} & \rotcolhead{$\ell_\subspace$} & \rotcolhead{$\sigma_{24}$} & \rotcolhead{$\ell_{24}$} & \rotcolhead{$\ell_{\subspace{}24}$} & \rotcolhead{$\sigma_{\subtime 1}$} & \rotcolhead{$\ell_{\subtime 1}$} & \rotcolhead{$\ell_{\subspace 1}$} & \rotcolhead{$\alpha_{\subtime 1}$} & \rotcolhead{$\sigma_{\subtime 2}$} & \rotcolhead{$\ell_{\subtime 2}$} & \rotcolhead{$\ell_{\subspace 2}$} & \rotcolhead{$\alpha_{\subtime 2}$} & \rotcolhead{$\sigma_{\subtime 3}$} & \rotcolhead{$\ell_{\subtime 3}$} & \rotcolhead{$\ell_{\subspace 3}$} & \rotcolhead{$\alpha_{\subtime 3}$}\\ 
\hline
$\kSESE$ & 0.4 & 3.7 & 2.7 & 176 &  &  &  &  &  &  &  &  &  &  &  &  &  &  & \\ 
$\kdiurn$ & 0.4 & 3.1 & 2.8 & 154 & 2.4 & 0.7 & 1414 &  &  &  &  &  &  &  &  &  &  &  & \\ 
$\ksumprod$ & 0.2 &  &  &  & 2.7 & 0.8 & 785 & 0.5 & 0.3 & 10 & 0.3 & 0.9 & 1.9 & 59 & 1.1 & 4.4 & 8.9 & 370 & 0.3\\ 


In [59]:
for param in rows
    if param == "\\hline"
        print(param)
        print("\n")
        continue
    end
    print(latex_param[param])
    print(" &\t")
    if units[param] != ""
        print("(")
        print(units[param])
        print(")")
    end
    for kern in kerns
        print(" &\t")
        print(get(param_dict[kern], param, ""))
    end
    print("\\\\ \n")
end

$\sigman$ &	($\degreeC$) &	0.4 &	0.4 &	0.2\\ 
\hline
$\sigma_\subtime$ &	($\degreeC$) &	3.7 &	3.1 &	\\ 
$\ell_\subtime$ &	($\mathrm{hr}$) &	2.7 &	2.8 &	\\ 
$\ell_\subspace$ &	($\mathrm{km}$) &	176 &	154 &	\\ 
\hline
$\sigma_{24}$ &	($\degreeC$) &	 &	2.4 &	2.7\\ 
$\ell_{24}$ &	($\mathrm{hr}$) &	 &	0.7 &	0.8\\ 
$\ell_{\subspace{}24}$ &	($\mathrm{km}$) &	 &	1414 &	785\\ 
\hline
$\sigma_{\subtime 1}$ &	($\degreeC$) &	 &	 &	0.5\\ 
$\ell_{\subtime 1}$ &	($\mathrm{hr}$) &	 &	 &	0.3\\ 
$\ell_{\subspace 1}$ &	($\mathrm{km}$) &	 &	 &	10\\ 
$\alpha_{\subtime 1}$ &	 &	 &	 &	0.3\\ 
\hline
$\sigma_{\subtime 2}$ &	($\degreeC$) &	 &	 &	0.9\\ 
$\ell_{\subtime 2}$ &	($\mathrm{hr}$) &	 &	 &	1.9\\ 
$\ell_{\subspace 2}$ &	($\mathrm{km}$) &	 &	 &	59\\ 
$\alpha_{\subtime 2}$ &	 &	 &	 &	1.1\\ 
\hline
$\sigma_{\subtime 3}$ &	($\degreeC$) &	 &	 &	4.4\\ 
$\ell_{\subtime 3}$ &	($\mathrm{hr}$) &	 &	 &	8.9\\ 
$\ell_{\subspace 3}$ &	($\mathrm{km}$) &	 &	 &	370\\ 
$\alpha_{\subtime 3}$ &	 &	 &	 &	0.3\\ 
\hline


In [77]:
fewer_rows = [
    "σ_eps",
    "\\hline",
    "σ_time",
    "ℓ_time",
    "ℓ_space",
    "α_time",
    "\\hline",
    "σ_24",
    "ℓ_24",
    "ℓ_s24",
]

10-element Array{String,1}:
 "σ_eps"  
 "\\hline"
 "σ_time" 
 "ℓ_time" 
 "ℓ_space"
 "α_time" 
 "\\hline"
 "σ_24"   
 "ℓ_24"   
 "ℓ_s24"  

In [84]:
for param in fewer_rows
    if param == "\\hline"
        print(param)
        print("\n")
        continue
    end
    print(latex_param[param])
    print(" &\t")
    if units[param] != ""
        print("(")
        print(units[param])
        print(")")
    end
    for kern in kerns
        print(" &\t")
        if param ∈ keys(param_dict[kern])
            print(param_dict[kern][param])
        elseif (param*"1") ∈ keys(param_dict[kern])
            for i in 1:3
                if i!=1
                    print(",\\,")
                end
                print(param_dict[kern][param*string(i)])
            end
        else
            print("\t")
        end
    end
    print("\\\\ \n")
end

$\sigman$ &	($\degreeC$) &	0.4 &	0.4 &	0.2\\ 
\hline
$\sigma_\subtime$ &	($\degreeC$) &	3.7 &	3.1 &	0.5,\,0.9,\,4.4\\ 
$\ell_\subtime$ &	($\mathrm{hr}$) &	2.7 &	2.8 &	0.3,\,1.9,\,8.9\\ 
$\ell_\subspace$ &	($\mathrm{km}$) &	176 &	154 &	10,\,59,\,370\\ 
$\alpha_{\subtime}$ &	 &		 &		 &	0.3,\,1.1,\,0.3\\ 
\hline
$\sigma_{24}$ &	($\degreeC$) &		 &	2.4 &	2.7\\ 
$\ell_{24}$ &	($\mathrm{hr}$) &		 &	0.7 &	0.8\\ 
$\ell_{\subspace{}24}$ &	($\mathrm{km}$) &		 &	1414 &	785\\ 
